In [1]:
import os
import json
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from datetime import datetime
from cog_analysis import load_boat_data
from IPython.display import display

def load_summary_intervals(summary_file="summary.json"):
    with open(summary_file, "r") as f:
        summary = json.load(f)
    return {r["run"]: r["intervals"] for r in summary}

def filter_interval(df, start, end):
    return df[(df["SecondsSince1970"] >= start) & (df["SecondsSince1970"] <= end)].reset_index(drop=True)

def compute_stats(df, columns):
    stats = {}
    for col in columns:
        data = df[col].dropna()
        if col == "COG":
            angles = np.deg2rad(data)
            mean = np.rad2deg(np.arctan2(np.mean(np.sin(angles)), np.mean(np.cos(angles)))) % 360
            stats[col] = {"Avg": mean, "Min": data.min(), "Max": data.max(), "Count": len(data), "StdDev": data.std()}
        else:
            stats[col] = {"Avg": data.mean(), "Min": data.min(), "Max": data.max(), "Count": len(data), "StdDev": data.std()}
    return pd.DataFrame(stats).T

def compute_distance(lat, lon):
    if len(lat) < 2:
        return 0
    return sum(geodesic((lat.iloc[i - 1], lon.iloc[i - 1]), (lat.iloc[i], lon.iloc[i])).meters 
               for i in range(1, len(lat)))

def compute_straight_line(lat, lon):
    if len(lat) < 2:
        return 0
    return geodesic((lat.iloc[0], lon.iloc[0]), (lat.iloc[-1], lon.iloc[-1])).meters

from math import radians, cos, sin

from math import radians, cos, sin
import numpy as np
import pandas as pd
def compute_directional_gain(df1, df2):
    """
    Compute relative gain with correct signs:
    - Positive values = Boat 1 gains on Boat 2
    - Negative values = Boat 1 loses to Boat 2
    Components:
    - Forward: boat's heading direction (COG)
    - Up: perpendicular to heading (port side)
    - VMG: true wind direction (TWD)
    """
    try:
        # Data validation
        if df1.empty or df2.empty or len(df1) < 2 or len(df2) < 2:
            raise ValueError("Insufficient data points")
        
        # Circular mean for angles
        def circular_mean(angles):
            rad = np.radians(angles)
            return np.degrees(np.arctan2(np.mean(np.sin(rad)), 
                            np.mean(np.cos(rad)))) % 360
            
        cog = circular_mean(df1["COG"].dropna())
        twd = circular_mean(df1["TWD"].dropna())
        
        # Unit vectors
        forward_vec = np.array([np.sin(np.radians(cog)), 
                              np.cos(np.radians(cog))])
        up_vec = np.array([-forward_vec[1], forward_vec[0]])  # Port side
        vmg_vec = np.array([np.sin(np.radians(twd)), 
                           np.cos(np.radians(twd))])
        
        # Position conversion (Boat 2 start as origin)
        def to_meters(lat, lon, ref_lat, ref_lon):
            x = (lon - ref_lon) * 111319.9 * np.cos(np.radians(ref_lat))
            y = (lat - ref_lat) * 111319.9
            return np.array([x, y])
            
        ref_lat, ref_lon = df2["Lat"].iloc[0], df2["Lon"].iloc[0]
        
        # Position vectors (Boat 1 relative to Boat 2)
        start1 = to_meters(df1["Lat"].iloc[0], df1["Lon"].iloc[0], ref_lat, ref_lon)
        start2 = to_meters(df2["Lat"].iloc[0], df2["Lon"].iloc[0], ref_lat, ref_lon)
        end1 = to_meters(df1["Lat"].iloc[-1], df1["Lon"].iloc[-1], ref_lat, ref_lon)
        end2 = to_meters(df2["Lat"].iloc[-1], df2["Lon"].iloc[-1], ref_lat, ref_lon)
        
        # Key correction: Calculate relative progress
        initial_rel_pos = start1 - start2  # Initial position difference
        final_rel_pos = end1 - end2        # Final position difference
        progress = initial_rel_pos - final_rel_pos  # Positive = gained
        
        # Projections
        def project(vec):
            return {
                'Forward': np.dot(vec, forward_vec),
                'Up': np.dot(vec, up_vec),
                'VMG': np.dot(vec, vmg_vec)
            }
        
        # Time duration in minutes
        duration = (df1["SecondsSince1970"].iloc[-1] - df1["SecondsSince1970"].iloc[0]) / 60
        
        # Results table
        return pd.DataFrame({
            'Forward': [
                -np.dot(initial_rel_pos, forward_vec),  # Initial deficit
                -np.dot(final_rel_pos, forward_vec),    # Final deficit
                np.dot(progress, forward_vec),          # Total gain
                np.dot(progress, forward_vec)/duration if duration > 0 else np.nan
            ],
            'Up': [
                -np.dot(initial_rel_pos, up_vec),
                -np.dot(final_rel_pos, up_vec),
                np.dot(progress, up_vec),
                np.dot(progress, up_vec)/duration if duration > 0 else np.nan
            ],
            'VMG': [
                -np.dot(initial_rel_pos, vmg_vec),
                -np.dot(final_rel_pos, vmg_vec),
                np.dot(progress, vmg_vec),
                np.dot(progress, vmg_vec)/duration if duration > 0 else np.nan
            ]
        }, index=['Initial Deficit', 'Final Deficit', 'Total Gain', 'Gain per Minute'])
        
    except Exception as e:
        print(f"Error: {str(e)}")
        return pd.DataFrame(
            [[np.nan, np.nan, np.nan]] * 4,
            index=['Initial Deficit', 'Final Deficit', 'Total Gain', 'Gain per Minute'],
            columns=['Forward', 'Up', 'VMG']
        )
"""
def compute_directional_gain(df1, df2):

    try:
        # Check for valid data
        if df1.empty or df2.empty or len(df1) < 2 or len(df2) < 2:
            raise ValueError("Not enough data points")
            
        if "COG" not in df1.columns or df1["COG"].isna().all():
            raise ValueError("Missing course over ground (COG) data")
            
        if "TWD" not in df1.columns or df1["TWD"].isna().all():
            raise ValueError("Missing true wind direction (TWD) data")

        # Get average boat heading (COG) and true wind direction
        cog = df1["COG"].mean()
        cog_rad = np.radians(cog)
        twd = df1["TWD"].mean()
        twd_rad = np.radians(twd)
        
        # Calculate unit vectors
        forward_vec = np.array([np.sin(cog_rad), np.cos(cog_rad)])  # Boat's forward direction
        down_vec = np.array([-forward_vec[1], forward_vec[0]])      # Perpendicular to heading (right side)
        vmg_vec = np.array([np.sin(twd_rad), np.cos(twd_rad)])      # Wind direction
        
        # Convert lat/lon to meters relative to starting point
        def latlon_to_meters(lat, lon, ref_lat, ref_lon):
            lat_diff = (lat - ref_lat) * 111319.9  # meters per degree latitude
            lon_diff = (lon - ref_lon) * 111319.9 * np.cos(np.radians(ref_lat))
            return np.array([lon_diff, lat_diff])
        
        # Use starting position of Boat 2 as reference point
        ref_lat = df2["Lat"].iloc[0]
        ref_lon = df2["Lon"].iloc[0]
        
        # Starting and ending positions in meters
        start1 = latlon_to_meters(df1["Lat"].iloc[0], df1["Lon"].iloc[0], ref_lat, ref_lon)
        start2 = latlon_to_meters(df2["Lat"].iloc[0], df2["Lon"].iloc[0], ref_lat, ref_lon)
        end1 = latlon_to_meters(df1["Lat"].iloc[-1], df1["Lon"].iloc[-1], ref_lat, ref_lon)
        end2 = latlon_to_meters(df2["Lat"].iloc[-1], df2["Lon"].iloc[-1], ref_lat, ref_lon)
        
        # Calculate gains (Boat 1 relative to Boat 2)
        initial_offset = start1 - start2
        final_offset = end1 - end2
        total_gain = final_offset - initial_offset
        
        # Project gains onto different axes
        def project(vec):
            return {
                'Forward': np.dot(vec, forward_vec),  # Along boat's heading
                'Down': np.dot(vec, down_vec),       # Perpendicular to heading
                'VMG': np.dot(vec, vmg_vec)         # Along wind direction
            }
        
        # Calculate time duration in minutes
        duration = (df1["SecondsSince1970"].iloc[-1] - df1["SecondsSince1970"].iloc[0]) / 60
        
        # Create results DataFrame
        results = pd.DataFrame({
            'Forward': [
                project(initial_offset)['Forward'],
                project(final_offset)['Forward'],
                project(total_gain)['Forward'],
                project(total_gain)['Forward']/duration if duration > 0 else np.nan
            ],
            'Down': [
                project(initial_offset)['Down'],
                project(final_offset)['Down'],
                project(total_gain)['Down'],
                project(total_gain)['Down']/duration if duration > 0 else np.nan
            ],
            'VMG': [
                project(initial_offset)['VMG'],
                project(final_offset)['VMG'],
                project(total_gain)['VMG'],
                project(total_gain)['VMG']/duration if duration > 0 else np.nan
            ]
        }, index=['Initial Offset', 'Final Offset', 'Total Gain', 'Gain per Minute'])
        
        return results
        
    except Exception as e:
        print(f"Error computing directional gain: {str(e)}")
        return pd.DataFrame(
            [[np.nan, np.nan, np.nan]] * 4,
            index=['Initial Offset', 'Final Offset', 'Total Gain', 'Gain per Minute'],
            columns=['Forward', 'Down', 'VMG']
        )
"""


def merge_stats(stats1, stats2, label1, label2):
    stats1 = stats1.rename(columns=lambda x: f"{x} ({label1})")
    stats2 = stats2.rename(columns=lambda x: f"{x} ({label2})")
    combined = pd.concat([stats1, stats2], axis=1)
    order = ["Avg", "Min", "Max", "Count", "StdDev"]
    cols = [f"{stat} ({label})" for stat in order for label in (label1, label2) if f"{stat} ({label})" in combined.columns]
    return combined[cols]

def display_all(merged_stats, summary_df, gain_table, boat1_name, boat2_name):
    # Tableau statistiques avec format numérique
    display(
        merged_stats.style
        .format("{:.4g}")
        .set_caption("Run tot Statistics")
    )

    display(
        summary_df.style
        .format("{:.4g}")
        .set_caption("Résumé des distances")
    )

    display(
        gain_table.style
        .set_caption(f"{boat1_name} gains relative to {boat2_name} in Meters")
    )


def load_and_reduce_boat_data(run_path, summary_dict):
    csv_files = sorted([f for f in os.listdir(run_path) if f.endswith(".csv")])
    if len(csv_files) < 2:
        raise ValueError("At least two CSV files are required.")

    boat1_df, boat2_df, boat1_name, boat2_name = load_boat_data(
        os.path.join(run_path, csv_files[0]),
        os.path.join(run_path, csv_files[1])
    )

    # Check if DataFrames are empty
    if boat1_df.empty or boat2_df.empty:
        raise ValueError("One or both boat DataFrames are empty")

    run_name = os.path.basename(run_path)
    if run_name not in summary_dict:
        raise ValueError(f"No intervals for run: {run_name}")
    intervals = summary_dict[run_name]
    if len(intervals) < 2:
        raise ValueError(f"Run {run_name} must have at least two intervals.")

    # Create filtered DataFrames
    boat1_int1 = filter_interval(boat1_df, intervals[0]["start_time"], intervals[0]["end_time"])
    boat2_int1 = filter_interval(boat2_df, intervals[0]["start_time"], intervals[0]["end_time"])
    boat1_int2 = filter_interval(boat1_df, intervals[1]["start_time"], intervals[1]["end_time"])
    boat2_int2 = filter_interval(boat2_df, intervals[1]["start_time"], intervals[1]["end_time"])

    # Check if filtered intervals are empty
    if boat1_int1.empty or boat2_int1.empty or boat1_int2.empty or boat2_int2.empty:
        raise ValueError("One or more interval DataFrames are empty")

    return {
        "full_df1": boat1_df,
        "full_df2": boat2_df,
        "reduced_boat1_int1_df": boat1_int1,
        "reduced_boat2_int1_df": boat2_int1,
        "reduced_boat1_int2_df": boat1_int2,
        "reduced_boat2_int2_df": boat2_int2,
        "boat1_name": boat1_name,
        "boat2_name": boat2_name
    }

def compare_runs(df1, df2, label1, label2):
    cols = ["TWS", "TWD", "SOG", "VMG", "COG", "TWA_Abs", "Heel_Lwd", "Trim"]
    stats1 = compute_stats(df1, cols)
    stats2 = compute_stats(df2, cols)
    dist1 = compute_distance(df1["Lat"], df1["Lon"])
    dist2 = compute_distance(df2["Lat"], df2["Lon"])
    straight1 = compute_straight_line(df1["Lat"], df1["Lon"])
    straight2 = compute_straight_line(df2["Lat"], df2["Lon"])
    summary_df = pd.DataFrame({label1: [dist1, straight1], label2: [dist2, straight2]}, index=["Distance [m]", "Straight Line [m]"])
    return stats1, stats2, summary_df

def process_run(df1, df2, name1, name2, title):
    if df1.empty or df2.empty:
        print(f"⚠️ Skipping {title} due to empty DataFrames: {name1} vs {name2}")
        return

    df1 = df1.copy()
    df2 = df2.copy()
    df1["ISODateTimeUTC"] = pd.to_datetime(df1["ISODateTimeUTC"], errors="coerce")
    df2["ISODateTimeUTC"] = pd.to_datetime(df2["ISODateTimeUTC"], errors="coerce")

    # Check for valid timestamps
    if df1["ISODateTimeUTC"].isna().all() or df2["ISODateTimeUTC"].isna().all():
        print(f"⚠️ Skipping {title} due to invalid timestamps: {name1} vs {name2}")
        return

    start_time = df1["ISODateTimeUTC"].iloc[0].ceil("min")
    df1 = df1[df1["ISODateTimeUTC"] >= start_time]
    df2 = df2[df2["ISODateTimeUTC"] >= start_time]

    if df1.empty or df2.empty:
        print(f"⚠️ Skipping {title} after time alignment: {name1} vs {name2}")
        return

    print("\n" + "=" * 80)
    print(f"📊 {title}: {name1} vs {name2}")
    print("=" * 80)

    stats1, stats2, summary_df = compare_runs(df1, df2, name1, name2)
    merged_stats = merge_stats(stats1, stats2, name1, name2)
    directional_gain = compute_directional_gain(df1, df2)
    display_all(merged_stats, summary_df, directional_gain, name1, name2)


def process_all_run(run_path, summary_path):
    summary_dict = load_summary_intervals(summary_path)
    result = load_and_reduce_boat_data(run_path, summary_dict)
    name1, name2 = result["boat1_name"], result["boat2_name"]

    process_run(result["full_df1"], result["full_df2"], name1, name2, "Full Interval")
    process_run(result["reduced_boat1_int1_df"], result["reduced_boat2_int1_df"], name1, name2, "Interval 1: Upwind")
    process_run(result["reduced_boat1_int2_df"], result["reduced_boat2_int2_df"], name1, name2, "Interval 2: Downwind")


In [2]:
import os
import json
from cog_analysis import analyze_session

summary_path = "summary.json"

base_dir = "../Data_Sailnjord/Straight_lines"

# Scan de tous les dossiers de date
for date_folder in sorted(os.listdir(base_dir)):
    date_path = os.path.join(base_dir, date_folder)

    # Scan de tous les runs dans chaque dossier de date
    for run_folder in sorted(os.listdir(date_path)):
        run_path = os.path.join(date_path, run_folder)
        print("\n\nProcessing:", run_path)
        try:
            process_all_run(run_path, summary_path)
        except ValueError as e:
            print(f"Error: {e}")



Processing: ../Data_Sailnjord/Straight_lines\06_06\06_06_Run1

📊 Full Interval: Gian Stragiotti vs Karl Maeder


,Avg (Gian Stragiotti),Avg (Karl Maeder),Min (Gian Stragiotti),Min (Karl Maeder),Max (Gian Stragiotti),Max (Karl Maeder),Count (Gian Stragiotti),Count (Karl Maeder),StdDev (Gian Stragiotti),StdDev (Karl Maeder)
TWS,7.388,7.388,5.204,5.204,9.5,9.5,3903,3903,1.021,1.021
TWD,176.9,176.9,166.6,166.6,195.3,195.3,3903,3903,7.709,7.709
SOG,18.53,18.74,0,5.6,28.3,27.4,3903,3903,5.83,4.469
VMG,10.73,10.43,0,0.003,20.2,19.98,3903,3903,5.034,4.351
COG,110.2,107.8,1,0.2,360,360,3903,3903,96.29,96.47
TWA_Abs,83.41,84.39,0.656,0.697,179.5,179.9,3903,3903,39.17,38.22
Heel_Lwd,40.04,40.4,-112.2,-26.4,95.6,76.1,3852,3839,22.23,16.19
Trim,11.91,7.739,-7.9,-7.7,49.3,42.8,3903,3903,5.829,4.889


,Gian Stragiotti,Karl Maeder
Distance [m],3737,3771
Straight Line [m],189.9,228.2


,Forward,Up,VMG
Initial Deficit,-17.622324,27.327566,-32.067308
Final Deficit,21.972680,25.259399,-14.519571
Total Gain,39.595005,-2.068167,17.547738
Gain per Minute,6.088245,-0.318008,2.698192



📊 Interval 1: Upwind: Gian Stragiotti vs Karl Maeder


...

,Forward,Up,VMG
Initial Deficit,-10.168994,-9.610727,0.786487
Final Deficit,-3.725766,-14.412579,8.611711
Total Gain,6.443227,-4.801851,7.825224
Gain per Minute,6.891398,-5.135853,8.369522



📊 Interval 2: Downwind: Karl Maeder vs SenseBoard


,Avg (Karl Maeder),Avg (SenseBoard),Min (Karl Maeder),Min (SenseBoard),Max (Karl Maeder),Max (SenseBoard),Count (Karl Maeder),Count (SenseBoard),StdDev (Karl Maeder),StdDev (SenseBoard)
TWS,8.101,8.101,7.705,7.705,8.7,8.7,222,222,0.264,0.2639
TWD,208,208,207.4,207.4,208.6,208.6,222,222,0.3219,0.3219
SOG,27.03,27.48,25.6,26.6,28.5,28.3,222,222,0.7799,0.5243
VMG,21.21,21.68,18.95,19.67,22.78,23.52,222,222,0.9375,0.8309
COG,349.7,350.1,345.2,343.7,355.6,356,222,222,2.084,2.402
TWA_Abs,141.8,142.1,136.9,136,148.2,148.2,222,222,2.213,2.241
Heel_Lwd,48.87,52.58,35.6,36.5,63.9,66.5,222,222,5.537,5.996
Trim,8.56,7.546,1.4,-1.6,19.4,14.5,222,222,3.295,2.941


,Karl Maeder,SenseBoard
Distance [m],307.5,312.8
Straight Line [m],307.3,312.5


,Forward,Up,VMG
Initial Deficit,23.350925,43.449911,8.553650
Final Deficit,28.635811,41.711644,3.326872
Total Gain,5.284887,-1.738267,-5.226777
Gain per Minute,14.346161,-4.718636,-14.188420
